# 将oneke_wrapper中常规推理更换为vllm推理并进行测试

In [1]:
import json
from oneke_wrapper import OneKEWrapper

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


安装了——ipywidgets-8.1.5 jupyterlab-widgets-3.0.13 widgetsnbextension-4.0.13

## 定义模型路径以及卡号

In [2]:
model_path = "./cache/models--baichuan-inc--Baichuan2-7B-Chat/snapshots/ea66ced17780ca3db39bc9f8aa601d8463db3da5"
lora_path = "./lora/baichuan7B-data2text-continue"

device = "cuda:3"

## 初始化类对象,加载模型

In [3]:
oneke_wrapper = OneKEWrapper(model_path, lora_path, device, False)

Initializing OneKEWrapper...


/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:556: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorc

## vllm推理单条测试

In [4]:
# 加载数据
data = json.load(open("./internal_data_type/data2text_test_all-v1.json"))

#选取第一条进行推理
oneke_wrapper.inference(data[0])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/lib/python3.10/contextlib.py:103: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


[{'table': {'header': ['时间',
    '人物',
    '国家',
    '装备名称',
    '装备类型',
    '装备数量',
    '组织机构',
    '地点',
    '任务',
    '行动',
    '事件'],
   'data': {'2': ['去年五月',
     '军方检察官',
     '台湾',
     '笔记本电脑',
     '导弹艇',
     '一台',
     '海军',
     '未明确',
     '未明确',
     '展开调查',
     '笔记本电脑丢失']}},
  'origin': {},
  'text': '一些消息人士透露，这两张地图中包含有关台湾海军舰艇布署的情况。如果发生战争，丢失这两张地图将使台湾海军在面对敌方时失去战斗力。此外，还有一台导弹艇上的笔记本电脑在去年五月丢失，军方检察官认真对此展开调查。',
  'verify': {'2': [{'v': '去年五月', 'span': '', 'cf': ''},
    {'v': None, 'span': '', 'cf': ''},
    {'v': '台湾', 'span': '', 'cf': ''},
    {'v': '笔记本电脑', 'span': '', 'cf': ''},
    {'v': None, 'span': '', 'cf': ''},
    {'v': '一台', 'span': '', 'cf': ''},
    {'v': '海军', 'span': '', 'cf': ''},
    {'v': None, 'span': '', 'cf': ''},
    {'v': None, 'span': '', 'cf': ''},
    {'v': '展开调查', 'span': '', 'cf': ''},
    {'v': '笔记本电脑丢失', 'span': '', 'cf': ''}]},
  'add_refs': []}]

## vllm推理全集测试，记录平均时间

In [5]:
# 所有数据计算平均时间
from tqdm import tqdm
oneke_wrapper.data_count = 0
oneke_wrapper.total_time = 0
results = []
for d in tqdm(data):
    results.append(oneke_wrapper.inference(d))
print(oneke_wrapper.total_time/oneke_wrapper.data_count)

 19%|█▉        | 409/2138 [24:07<1:39:09,  3.44s/it]

Expecting ',' delimiter: line 1 column 69 (char 68)
input:{'instruction': '你是一个命名实体识别专家。请从input中抽取符合schema描述的实体，如果实体类型不存在就返回空列表，并输出为可解析的json格式。', 'schema': ['时间', '人物', '国家', '装备名称', '装备类型', '装备数量', '组织机构', '地点', '任务', '行动', '事件'], 'input': '美国海军第七舰队旗舰「小鹰」号航空母舰于10月13日抵达北海道小樽市的小樽港，登陆日本民用港口是美日两国新防卫合作指针和「周边事态法」的成果之一，遭到担忧小樽朝军港化发展的民间组织在港口集结抗议，但航母仍然在胜纳埠头安然停泊。这艘满载排水量81,775吨的「小鹰」号是继1997年9月停泊于此的航母「独立」号之后第二艘进入小樽港的美国航空母舰，这也是美国军舰首次进入日本民用港口。'}, output:{"时间": ["10月13日"], "人物": [], "国家": ["美国"], "装备名称": ["小鹰」号, «独立」号], "装备类型": ["航空母舰"], "装备数量": ["1"], "组织机构": [], "地点": ["北海道小樽市的小樽港"], "任务": [], "行动": [], "事件": []}


 30%|██▉       | 633/2138 [37:40<1:41:01,  4.03s/it]

Expecting ',' delimiter: line 1 column 130 (char 129)
input:{'instruction': '你是一个命名实体识别专家。请从input中抽取符合schema描述的实体，如果实体类型不存在就返回空列表，并输出为可解析的json格式。', 'schema': ['时间', '人物', '国家', '装备名称', '装备类型', '装备数量', '组织机构', '地点', '任务', '行动', '事件'], 'input': '俄罗斯签署了两份建造合同，其中包括6艘11356型护卫舰为黑海舰队建造。第一艘格里戈洛维奇上将号已于3月14日下水，另外5艘正在建造中。根据合同规定，这批护卫舰将在2014年至2017年期间陆续交付黑海舰队。'}, output:{"时间": ["3月14日", "2014年至2017年"], "人物": [], "国家": ["俄罗斯"], "装备名称": ["格里戈洛维奇上将号", "11356型护卫舰"], "装备类型": ["护卫舰"], "装备数量": ["1", "6"]， "组织机构": ["黑海舰队"], "地点": ["黑海"], "任务": [], "行动": [], "事件": []}


100%|██████████| 2138/2138 [2:10:17<00:00,  3.66s/it]  

3.6507059208462747


In [6]:
print(f"Average inference time: {oneke_wrapper.total_time/oneke_wrapper.data_count}")

Average inference time: 3.6507059208462747


In [14]:
import torch
memory_used = torch.cuda.memory_allocated(device=3) - oneke_wrapper.gpu_mem_before_load
memory_occupied = torch.cuda.memory_allocated(device=3) / oneke_wrapper.gpu_mem_max * 100
print(f"Memory used for inference: {memory_used / (1024 ** 2):.2f} MB")
print(f"Memory occupied after inference: {memory_occupied:.2f}%")

15740968960 0 25430786048
Memory used for inference: 15011.76 MB
Memory occupied after inference: 61.90%


In [8]:
print(len(results))

2138


In [9]:
json.dump(results, open("./results/baichuan.json",'w+'),ensure_ascii = False)